# Segmentation and clustering project part 1

Installing Packages 

In [7]:
pip install wikipedia


Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install -U setuptools

  Attempting uninstall: setuptools
    Found existing installation: setuptools 45.2.0.post20200210
    Uninstalling setuptools-45.2.0.post20200210:
      Successfully uninstalled setuptools-45.2.0.post20200210
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


**Importing Packages**

In [18]:
import pandas as pd
import wikipedia as wk
from bs4 import BeautifulSoup

In [19]:
#importing the wikipedia page into html format
html = wk.page("List of postal codes of Canada: M").html().encode('UTF-8')

In [20]:
#Converting the html file into dataframe using pandas
df = pd.read_html(html, header=0)[0]

In [21]:
#Removing rows which have Borough as Not assigned
df = df[df.Borough != 'Not assigned']

In [22]:
#Combining Neighborhoods with same postal code and borough
df = df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(list).apply(lambda x: ', '.join(x)).to_frame().reset_index()

In [23]:
#if any neighborhood as Not assigned, then it is assigned the name of Borough
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

In [24]:
df.shape
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [25]:
#Adding the latitude and longitude to every postal codes using geospatial_coordinate csv
import requests
import io
url = "http://cocl.us/Geospatial_data"
s = requests.get(url).content

In [26]:
ll = pd.read_csv(r'C:\Users\PRK\Desktop\Geospatial_Coordinates.csv')

In [27]:
df = pd.merge(df, ll, on = 'Postal Code')

In [28]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [2]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\PRK\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0
  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m

The following packages will b



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda




In [6]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: ...working... done

# All requested packages already installed.



In [13]:
#Getting the coordinates of toronto for the folium map as center
address = 'Toronto, ON'
geolocator = Nominatim(user_agent='to_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude, 'are coord of toronto')

43.6534817 -79.3839347 are coord of toronto


In [55]:
#creating a folium map with the coord of toronto
map = folium.Map(location=[latitude, longitude], zoom_start=10)

#creating the points of lat and long of every postal code in the foliom map with popup label as description 
#of borough and neighborhood
for lat, lon, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{},{}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color = 'blue',
        fill=True,
        fill_color='black',
        fill_opacity=0.55,
        parse_html=True).add_to(map)



In [56]:
map